In [1]:
import os
path = os.getcwd()

11 应用系统负载分析与磁盘容量预测：时序分析

11-1 数据预处理：属性构造

In [2]:
import pandas as pd

In [3]:
discfile = path + '/data/discdata.xls'
transformeddata = path + '/tmp/discdata_processed.xls'
data = pd.read_excel(discfile)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 7 columns):
SYS_NAME       188 non-null object
NAME           188 non-null object
TARGET_ID      188 non-null int64
DESCRIPTION    188 non-null object
ENTITY         188 non-null object
VALUE          188 non-null float64
COLLECTTIME    188 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 10.4+ KB


In [4]:
data.head()

,SYS_NAME,NAME,TARGET_ID,DESCRIPTION,ENTITY,VALUE,COLLECTTIME
0,财务管理系统,CWXT_DB,184,磁盘已使用大小,C:\,3.427079e+07,2014-10-01
1,财务管理系统,CWXT_DB,184,磁盘已使用大小,D:\,8.026259e+07,2014-10-01
2,财务管理系统,CWXT_DB,183,磁盘容量,C:\,5.232332e+07,2014-10-01
3,财务管理系统,CWXT_DB,183,磁盘容量,D:\,1.572833e+08,2014-10-01
4,财务管理系统,CWXT_DB,184,磁盘已使用大小,C:\,3.432890e+07,2014-10-02


In [5]:
# 提取某部分数据
data = data[data['TARGET_ID'] == 184].copy()
# 以时间分组
data_group = data.groupby('COLLECTTIME')
data_group.head()

,SYS_NAME,NAME,TARGET_ID,DESCRIPTION,ENTITY,VALUE,COLLECTTIME
0,财务管理系统,CWXT_DB,184,磁盘已使用大小,C:\,34270787.33,2014-10-01
1,财务管理系统,CWXT_DB,184,磁盘已使用大小,D:\,80262592.65,2014-10-01
4,财务管理系统,CWXT_DB,184,磁盘已使用大小,C:\,34328899.02,2014-10-02
5,财务管理系统,CWXT_DB,184,磁盘已使用大小,D:\,83200151.65,2014-10-02
8,财务管理系统,CWXT_DB,184,磁盘已使用大小,C:\,34327553.50,2014-10-03
9,财务管理系统,CWXT_DB,184,磁盘已使用大小,D:\,83208320.00,2014-10-03
12,财务管理系统,CWXT_DB,184,磁盘已使用大小,C:\,34288672.21,2014-10-04
13,财务管理系统,CWXT_DB,184,磁盘已使用大小,D:\,83099271.65,2014-10-04
16,财务管理系统,CWXT_DB,184,磁盘已使用大小,C:\,34190978.41,2014-10-05
17,财务管理系统,CWXT_DB,184,磁盘已使用大小,D:\,82765171.65,2014-10-05


In [6]:
# 属性变换,改变列名
def attr_trans(x):
    result = pd.Series(
        index=['SYS_NAME',
               'CWXT_DB:184:C:\\',
               'CWXT_DB:184:D:\\',
               'COLLECTTIME']
    )
    result['SYS_NAME'] = x['SYS_NAME'].iloc[0]
    result['COLLECTTIME'] = x['COLLECTTIME'].iloc[0]
    result['CWXT_DB:184:C:\\'] = x['VALUE'].iloc[0]
    result['CWXT_DB:184:D:\\'] = x['VALUE'].iloc[1]
    return result

# 逐组处理
data_processed = data_group.apply(attr_trans)
data_processed.to_excel(transformeddata, index = False)
data_processed.head()

,SYS_NAME,CWXT_DB:184:C:\,CWXT_DB:184:D:\,COLLECTTIME
COLLECTTIME,,,,
2014-10-01,财务管理系统,34270787.33,80262592.65,2014-10-01
2014-10-02,财务管理系统,34328899.02,83200151.65,2014-10-02
2014-10-03,财务管理系统,34327553.50,83208320.00,2014-10-03
2014-10-04,财务管理系统,34288672.21,83099271.65,2014-10-04
2014-10-05,财务管理系统,34190978.41,82765171.65,2014-10-05


11-2 模型构建

11-2-1 平稳性检验

In [7]:
from statsmodels.tsa.stattools import adfuller as ADF

data_processed = data_processed.iloc[:len(data) - 5]    # 去除最后5个数据

# 平稳性检测
diff = 0
adf = ADF(data_processed['CWXT_DB:184:D:\\'])
while adf[1] > 0.05:  # p值为adf[1]，p值小于0.05认为是平稳的
    diff = diff + 1
    adf = ADF(data_processed['CWXT_DB:184:D:\\'].diff(diff).dropna())
print(u'原始序列经过%s阶差分后归于平稳，p值为%s' % (diff, adf[1]))

原始序列经过1阶差分后归于平稳，p值为2.7020610915301862e-06


11-2-2 白噪声检验

In [8]:
from statsmodels.stats.diagnostic import acorr_ljungbox

# 原始序列白噪声检验
[[lb], [p]] = acorr_ljungbox(data_processed['CWXT_DB:184:D:\\'], lags = 1)
if p < 0.05:
    print(u'原始序列为非白噪声序列，对应的p值为：%s' % p)
else:
    print(u'原始序列为白噪声序列，对应的p值为：%s' % p)

原始序列为非白噪声序列，对应的p值为：4.5840718221565076e-07


In [9]:
# 一阶差分序列白噪声检验
[[lb], [p]] = acorr_ljungbox(data_processed['CWXT_DB:184:D:\\'].diff().dropna(), lags = 1)
if p < 0.05:
    print(u'一阶差分序列为非白噪声序列，对应的p值为：%s' % p)
else:
    print(u'一阶差分序列为白噪声序列，对应的p值为：%s' % p)
print(lb)

一阶差分序列为白噪声序列，对应的p值为：0.0977263412698132
2.742262038851296


11-3 建模识别

11-3-1 确定模型ARIMA最优参数p、d、q

In [10]:
xdata = data_processed['CWXT_DB:184:D:\\']
xdata.index

DatetimeIndex(['2014-10-01', '2014-10-02', '2014-10-03', '2014-10-04',
               '2014-10-05', '2014-10-06', '2014-10-07', '2014-10-08',
               '2014-10-09', '2014-10-10', '2014-10-11', '2014-10-12',
               '2014-10-13', '2014-10-14', '2014-10-15', '2014-10-16',
               '2014-10-17', '2014-10-18', '2014-10-19', '2014-10-20',
               '2014-10-21', '2014-10-22', '2014-10-23', '2014-10-24',
               '2014-10-25', '2014-10-26', '2014-10-27', '2014-10-28',
               '2014-10-29', '2014-10-30', '2014-10-31', '2014-11-01',
               '2014-11-02', '2014-11-03', '2014-11-04', '2014-11-05',
               '2014-11-06', '2014-11-07', '2014-11-08', '2014-11-09',
               '2014-11-10', '2014-11-11', '2014-11-12', '2014-11-13',
               '2014-11-14', '2014-11-15', '2014-11-16'],
              dtype='datetime64[ns]', name='COLLECTTIME', freq='D')

In [11]:
xdata.describe()

count    4.700000e+01
mean     8.477504e+07
std      2.115162e+06
min      8.026259e+07
25%      8.320424e+07
50%      8.450061e+07
75%      8.630771e+07
max      8.976660e+07
Name: CWXT_DB:184:D:\, dtype: float64

In [12]:
from statsmodels.tsa.arima_model import ARIMA
# 定阶
pmax = int(len(xdata) / 10)
qmax = int(len(xdata) / 10)
# 定义bic矩阵
bic_matrix = []
for p in range(pmax + 1):
    tmp = []
    for q in range(qmax + 1):
        try:
            tmp.append(ARIMA(xdata, (p, 1, q)).fit().bic)
        except:
            tmp.append(None)
    bic_matrix.append(tmp)
bic_matrix = pd.DataFrame(bic_matrix)
bic_matrix

D:\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:650: RuntimeWarning: invalid value encountered in true_divide
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
D:\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:651: RuntimeWarning: invalid value encountered in true_divide
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
D:\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:607: RuntimeWarning: invalid value encountered in true_divide
  (1+np.exp(-params))).copy()
D:\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:609: RuntimeWarning: invalid value encountered in true_divide
  (1+np.exp(-params))).copy()
D:\Anaconda3\lib\site-packages\statsmodels\base\model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
D:\Anaconda3\lib\site-packages\statsmodels\base\model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'avail

,0,1,2,3,4
0,1431.648944,1428.768474,1427.757807,1429.661889,None
1,1432.582681,1426.961590,NaN,NaN,None
2,1435.644365,NaN,NaN,NaN,None
3,1433.912779,1433.500637,1437.113241,1441.530168,None
4,1436.607483,1435.793552,1440.912949,1444.221219,None


In [13]:
bic = bic_matrix.stack().astype('float64')
bic

0  0    1431.648944
   1    1428.768474
   2    1427.757807
   3    1429.661889
1  0    1432.582681
   1    1426.961590
2  0    1435.644365
3  0    1433.912779
   1    1433.500637
   2    1437.113241
   3    1441.530168
4  0    1436.607483
   1    1435.793552
   2    1440.912949
   3    1444.221219
dtype: float64

In [14]:
p, q = bic.idxmin()    # 展平，找出最小值
print(u'BIC最小的p值和q值为：%s、%s' % (p, q))

BIC最小的p值和q值为：1、1


11-3-2 构建模型

In [15]:
model = ARIMA(xdata, (p, 1, q)).fit()     # 建立ARIMA(p, d, q)模型，其中d为差分阶数
print(u'模型报告：\n', model.summary2())  # 给出一份模型报告
print(u'预测结果：\n', model.forecast(7)) # 作为周期7天的预测，返回预测结果、标准误差、置信区间

模型报告：
                                    Results: ARIMA
Model:                   ARIMA                  BIC:                      1426.9616  
Dependent Variable:      D.CWXT_DB:184:D:\      Log-Likelihood:           -705.82    
Date:                    2018-08-03 17:21       Scale:                    1.0000     
No. Observations:        46                     Method:                   css-mle    
Df Model:                3                      Sample:                   10-02-2014 
Df Residuals:            43                                               11-16-2014 
Converged:               1.0000                 S.D. of innovations:      1082513.061
No. Iterations:          62.0000                HQIC:                     1422.387   
AIC:                     1419.6470                                                   
-------------------------------------------------------------------------------------
                           Coef.     Std.Err.     t     P>|t|    [0.025      0.975]

11-3-3 模型检验

In [16]:
lagnum = 12    # 残差延迟个数

# 预测并计算残差
xdata_pred = model.predict(typ = 'levels')
pred_error = (xdata_pred - xdata).dropna()

lb, p = acorr_ljungbox(pred_error, lags = lagnum)
h = (p < 0.05).sum()    # p值小于0.05，则为非白噪声
if h > 0:
    print(u'模型ARIMA（1,1,1)不符合白噪声检验')
else:
    print(u'模型ARIMA（1,1,1)符合白噪声检验')
print(lb)

模型ARIMA（1,1,1)符合白噪声检验
[ 0.14465854  0.14935311  1.58882413  1.64419681  1.65436039  1.65710241
  4.53693463  5.05999471  6.75701363  8.84773537 11.32172459 11.33375405]


11-4 模型评价

In [17]:
file = path + '/data/predictdata.xls'
data_pred = pd.read_excel(file)
data_pred.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2014-11-12 to 2014-11-16
Data columns (total 2 columns):
预测值    5 non-null float64
实际值    5 non-null float64
dtypes: float64(2)
memory usage: 120.0 bytes


In [18]:
data_pred.head()

,预测值,实际值
2014-11-12,88.034303,87.249336
2014-11-13,88.217008,86.986142
2014-11-14,88.399714,86.678240
2014-11-15,88.582420,89.766600
2014-11-16,88.765125,89.377527


In [19]:
data_pred.columns

Index(['预测值', '实际值'], dtype='object')

In [34]:
# 计算误差
abs_ = (data_pred[u'预测值'] - data_pred[u'实际值']).abs()  
mae = abs_.mean()                                # 平均绝对误差
rmse = ((abs_ ** 2).mean()) ** 0.5               # 均方根误差
mape = (abs_ / data_pred[u'实际值']).mean()      # 平均绝对百分误差

print(
    u'''
    平均绝对误差为：    %0.4f,
    均方根误差为:       %0.4f,
    平均绝对百分误差为：%0.6f
    '''
    % (mae, rmse, mape)
)


    平均绝对误差为：    1.1068,
    均方根误差为:       1.1723,
    平均绝对百分误差为：0.012610
    
